### Queries with Joins on VAERS Dataset

In [1]:
# Remove limit on number of columns shown
import pandas as pd
pd.set_option('display.max_columns', None)

#### Inner Join by ID (between Patient_Event and Vaccine_Event tables) to identify details of vaccine administered to individuals who died as result of adverse event after vaccination, ordered by death date

In [2]:
%%bigquery
select evn.DATEDIED, vac.VAX_DATE, vac.VAX_NAME, vac.VAX_TYPE, vac.VAX_MANU
from vaers_modeled.Patient_Event evn
join vaers_modeled.Vaccine_Event vac on evn.VAERS_ID = vac.VAERS_ID
where evn.DIED = true
order by evn.DATEDIED desc
limit 12

,DATEDIED,VAX_DATE,VAX_NAME,VAX_TYPE,VAX_MANU
0,2019-12-08,2019-11-11,INFLUENZA (SEASONAL) (AFLURIA),FLU3,CSL LIMITED
1,2019-11-27,2019-11-27,DTAP + HEPB + IPV (PEDIARIX),DTAPHEPBIP,GLAXOSMITHKLINE BIOLOGICALS
2,2019-11-18,2019-11-12,ZOSTER (SHINGRIX),VARZOS,GLAXOSMITHKLINE BIOLOGICALS
3,2019-11-18,2019-11-12,ZOSTER (SHINGRIX),VARZOS,GLAXOSMITHKLINE BIOLOGICALS
4,2019-11-18,2019-11-12,ZOSTER (SHINGRIX),VARZOS,GLAXOSMITHKLINE BIOLOGICALS
5,2019-11-13,2019-01-11,HEP B (RECOMBIVAX HB),HEP,MERCK & CO. INC.
6,2019-11-13,2019-01-11,ROTAVIRUS (ROTATEQ),RV5,MERCK & CO. INC.
7,2019-11-13,2019-01-11,DTAP + IPV + HIB (PENTACEL),DTAPIPVHIB,SANOFI PASTEUR
8,2019-11-13,2019-01-11,DTAP + IPV + HIB (PENTACEL),DTAPIPVHIB,SANOFI PASTEUR
9,2019-11-13,2019-01-11,VACCINE NOT SPECIFIED (NO BRAND NAME),UNK,UNKNOWN MANUFACTURER


#### Inner Join by ID (between Patient_Event and Vaccine_Event tables) to identify anatomic site of vaccine injection (VAX_ROUTE) of individuals who died as result of adverse event, ordered by vaccination date

##### Legend for anatomic site of vaccine injection (VAX_ROUTE): 
IM - Intramuscular (Into muscle)\
SC - Subcutaneous (Under skin) \
SYR - Syringe (Not specified futher) 

In [3]:
%%bigquery
select VAX_DATE, ONSET_DATE, VAX_ROUTE 
from vaers_modeled.Patient_Event evn
join vaers_modeled.Vaccine_Event vac on evn.VAERS_ID = vac.VAERS_ID
where evn.DIED = true
order by vac.VAX_DATE desc
limit 12

,VAX_DATE,ONSET_DATE,VAX_ROUTE
0,2019-11-27,2019-11-27,IM
1,2019-11-12,2019-11-13,SC
2,2019-11-12,2019-11-13,SC
3,2019-11-12,2019-11-13,SC
4,2019-11-11,2019-11-12,IM
5,2019-11-07,2019-11-08,IM
6,2019-11-04,2019-11-05,IM
7,2019-10-29,2019-10-29,IM
8,2019-10-29,2019-10-29,IM
9,2019-10-23,2019-10-23,SYR


#### Full Outer Join by ID (between Vaccines and Symptoms tables) to determine vaccines which resulted in adverse symptoms of blindness among patients, ordered by vaccine name

In [4]:
%%bigquery
select VAX_NAME, VAX_TYPE, VAX_MANU
from vaers_staging.Vaccines vac 
full outer join vaers_staging.Symptoms sym on vac.VAERS_ID = sym.VAERS_ID
where SYMPTOM1 = 'Blindness' or SYMPTOM2 = 'Blindness' or SYMPTOM3 = 'Blindness'
order by vac.VAX_NAME 
limit 12

,VAX_NAME,VAX_TYPE,VAX_MANU
0,HEP A (HAVRIX),HEPA,GLAXOSMITHKLINE BIOLOGICALS
1,HEP A (HAVRIX),HEPA,GLAXOSMITHKLINE BIOLOGICALS
2,HEP A + HEP B (TWINRIX),HEPAB,GLAXOSMITHKLINE BIOLOGICALS
3,HPV (GARDASIL 9),HPV9,MERCK & CO. INC.
4,HPV (GARDASIL 9),HPV9,MERCK & CO. INC.
5,HPV (GARDASIL 9),HPV9,MERCK & CO. INC.
6,INFLUENZA (SEASONAL) (FLUAD),FLUA3,NOVARTIS VACCINES AND DIAGNOSTICS
7,INFLUENZA (SEASONAL) (FLUARIX QUADRIVALENT),FLU4,GLAXOSMITHKLINE BIOLOGICALS
8,INFLUENZA (SEASONAL) (FLULAVAL QUADRIVALENT),FLU4,GLAXOSMITHKLINE BIOLOGICALS
9,INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE),FLU3,SANOFI PASTEUR


#### Redone code after modeling data using Patient_Event which now includes Symptoms.

In [5]:
%%bigquery
select VAX_NAME, VAX_TYPE, VAX_MANU
from vaers_modeled.Vaccine_Event vac 
full outer join vaers_modeled.Patient_Event pat on vac.VAERS_ID = pat.VAERS_ID
where SYMPTOM1 = 'Blindness' or SYMPTOM2 = 'Blindness' or SYMPTOM3 = 'Blindeness'
order by vac.VAX_NAME 
limit 12

,VAX_NAME,VAX_TYPE,VAX_MANU
0,HEP A (HAVRIX),HEPA,GLAXOSMITHKLINE BIOLOGICALS
1,HEP A (HAVRIX),HEPA,GLAXOSMITHKLINE BIOLOGICALS
2,HEP A + HEP B (TWINRIX),HEPAB,GLAXOSMITHKLINE BIOLOGICALS
3,HPV (GARDASIL 9),HPV9,MERCK & CO. INC.
4,HPV (GARDASIL 9),HPV9,MERCK & CO. INC.
5,HPV (GARDASIL 9),HPV9,MERCK & CO. INC.
6,INFLUENZA (SEASONAL) (FLUAD),FLUA3,NOVARTIS VACCINES AND DIAGNOSTICS
7,INFLUENZA (SEASONAL) (FLUARIX QUADRIVALENT),FLU4,GLAXOSMITHKLINE BIOLOGICALS
8,INFLUENZA (SEASONAL) (FLULAVAL QUADRIVALENT),FLU4,GLAXOSMITHKLINE BIOLOGICALS
9,INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE),FLU3,SANOFI PASTEUR


#### Full Outer Join by ID (between Symptoms and Vaccines tables) to investigate adverse symptoms as result of HPV vaccine ordered by ID

In [6]:
%%bigquery
select SYMPTOM1, SYMPTOM2, SYMPTOM3
from vaers_staging.Symptoms sym 
full outer join vaers_staging.Vaccines vac on vac.VAERS_ID = sym.VAERS_ID
where VAX_TYPE = 'HPVX'
order by vac.VAERS_ID desc
limit 12

,SYMPTOM1,SYMPTOM2,SYMPTOM3
0,Erythema,Skin warm,None
1,Precancerous cells present,Smear cervix abnormal,None
2,Lip discolouration,None,None
3,Syncope,None,None
4,Dermatitis acneiform,Rash,Rash papular
5,Loss of personal independence in daily activities,Malaise,Oral herpes
6,Chills,Fatigue,Hyperhidrosis
7,Pyrexia,None,None
8,Headache,Pyrexia,None
9,Extra dose administered,Wrong product administered,None


#### Redone code using modeled Patient_Events which contains Symptoms.

In [7]:
%%bigquery
select SYMPTOM1, SYMPTOM2, SYMPTOM3
from vaers_modeled.Patient_Event pat 
full outer join vaers_modeled.Vaccine_Event vac on vac.VAERS_ID = pat.VAERS_ID
where VAX_TYPE = 'HPVX'
order by vac.VAERS_ID desc
limit 12

,SYMPTOM1,SYMPTOM2,SYMPTOM3
0,Erythema,Skin warm,None
1,Precancerous cells present,Smear cervix abnormal,None
2,Lip discolouration,None,None
3,Syncope,None,None
4,Dermatitis acneiform,Rash,Rash papular
5,Loss of personal independence in daily activities,Malaise,Oral herpes
6,Chills,Fatigue,Hyperhidrosis
7,Pyrexia,None,None
8,Headache,Pyrexia,None
9,Extra dose administered,Wrong product administered,None


#### Full Outer Join by ID (between Symptoms and Events tables) to investigate adverse symptoms of patients who had an ER visit and were diagnosed with life-threatening disease ordered by vaccination date

In [8]:
%%bigquery
select SYMPTOM1, SYMPTOM2, SYMPTOM3
from vaers_staging.Symptoms sym
full outer join vaers_staging.Events evn on sym.VAERS_ID = evn.VAERS_ID
where evn.ER_VISIT = true and evn.L_THREAT = true
order by evn.VAX_DATE desc

,SYMPTOM1,SYMPTOM2,SYMPTOM3
0,Dysphagia,Electrocardiogram,Sensation of foreign body
1,Urine output decreased,Vomiting,None
2,Barium enema,Dehydration,Diarrhoea haemorrhagic
3,Intussusception,Lid sulcus deepened,Procedural failure
4,Restlessness,Tic,Tongue movement disturbance
5,CSF red blood cell count positive,CSF test,Dystonia
6,Obstructive airways disorder,Opisthotonus,Prothrombin time normal
7,Muscle twitching,Muscular weakness,Musculoskeletal stiffness
8,Full blood count normal,Infantile back arching,Insomnia
9,Activated partial thromboplastin time normal,Apparent life threatening event,Arthropathy


#### Modeled tables have joined features of Events and Symptoms into one table Patient_Event so above join is mostly obsolete. Still need to join on Vaccine_Event in order to get VAX_DATE

In [9]:
%%bigquery
select SYMPTOM1, SYMPTOM2, SYMPTOM3
from vaers_modeled.Patient_Event pat
full outer join vaers_modeled.Vaccine_Event evn on pat.VAERS_ID = evn.VAERS_ID
where pat.ER_VISIT = true and pat.L_THREAT = true
order by evn.VAX_DATE desc
limit 12

,SYMPTOM1,SYMPTOM2,SYMPTOM3
0,Dysphagia,Electrocardiogram,Sensation of foreign body
1,Barium enema,Dehydration,Diarrhoea haemorrhagic
2,Barium enema,Dehydration,Diarrhoea haemorrhagic
3,Barium enema,Dehydration,Diarrhoea haemorrhagic
4,Intussusception,Lid sulcus deepened,Procedural failure
5,Urine output decreased,Vomiting,None
6,Urine output decreased,Vomiting,None
7,Urine output decreased,Vomiting,None
8,Intussusception,Lid sulcus deepened,Procedural failure
9,Intussusception,Lid sulcus deepened,Procedural failure


#### Inner Join by ID (between Events, Vaccines and Symptoms tables) to investigate type of vaccine and adverse symptoms of patients who recovered from adverse events in the state of Texas

In [10]:
%%bigquery
select VAX_NAME, VAX_TYPE, SYMPTOM1, SYMPTOM2, SYMPTOM3
from (vaers_staging.Events evn join vaers_staging.Vaccines vac on evn.VAERS_ID = vac.VAERS_ID)
    join vaers_staging.Symptoms sym on sym.VAERS_ID = evn.VAERS_ID
where RECOVD = 'Y' and STATE = 'TX'
limit 12

,VAX_NAME,VAX_TYPE,SYMPTOM1,SYMPTOM2,SYMPTOM3
0,HEP A (VAQTA),HEPA,Peripheral coldness,None,None
1,HEP A (VAQTA),HEPA,Blood glucose decreased,Cold sweat,Dizziness
2,HIB (HIBERIX),HIBV,Rash,Screaming,None
3,HIB (HIBERIX),HIBV,Cognitive disorder,Disturbance in attention,Hyperaesthesia
4,HEP A (HAVRIX),HEPA,Erythema,Pain,Secretion discharge
5,HEP A (HAVRIX),HEPA,Syncope,Unresponsive to stimuli,None
6,HEP A (HAVRIX),HEPA,Blood pressure fluctuation,Dizziness,Fall
7,DTAP (INFANRIX),DTAP,Erythema,Pain,Secretion discharge
8,DTAP (INFANRIX),DTAP,Rash,Screaming,None
9,DTAP (INFANRIX),DTAP,Cognitive disorder,Disturbance in attention,Hyperaesthesia


#### Join above more simple with modeled tables. Need only one join between both tables

In [11]:
%%bigquery
select VAX_NAME, VAX_TYPE, SYMPTOM1, SYMPTOM2, SYMPTOM3
from vaers_modeled.Patient_Event evn join vaers_modeled.Vaccine_Event vac on evn.VAERS_ID = vac.VAERS_ID
where evn.RECOVD = 'Y' and evn.STATE = 'TX'
limit 12

,VAX_NAME,VAX_TYPE,SYMPTOM1,SYMPTOM2,SYMPTOM3
0,ZOSTER (SHINGRIX),VARZOS,Tremor,None,None
1,ZOSTER (SHINGRIX),VARZOS,Pain in extremity,None,None
2,MENINGOCOCCAL CONJUGATE (MENVEO),MNQ,Wrong product administered,None,None
3,MENINGOCOCCAL CONJUGATE (MENVEO),MNQ,Pain,Pyrexia,None
4,DTAP + IPV (KINRIX),DTAPIPV,Injection site erythema,Injection site swelling,Injection site warmth
5,MEASLES + MUMPS + RUBELLA + VARICELLA (PROQUAD),MMRV,Injection site erythema,Injection site swelling,Injection site warmth
6,PNEUMO (PNEUMOVAX),PPV,Erythema,Injection site erythema,Injection site pain
7,TDAP (ADACEL),TDAP,Expired product administered,None,None
8,INFLUENZA (SEASONAL) (FLUARIX QUADRIVALENT),FLU4,Injection site pain,None,None
9,HIB (ACTHIB),HIBV,Irritability,Rash,Urticaria
